# General Imports

In [1]:
import os
import time
import re
import warnings
import gc

gc.collect()
warnings.filterwarnings('ignore')

# General Tasks

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

google_search_project_api_key = os.getenv('google_search_api_key')
google_search_project_id = os.getenv('google_search_project_id')
google_gemini_api_key = os.getenv('google_gemini_api_key')


# LLM

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
        model='gemini-1.5-pro',
        temperature=0.9,
        google_api_key=google_gemini_api_key,
    )

gc.collect()

20

In [4]:
response = llm.invoke('how many calories are in 100 grams of wheat flour roti in asia?(Give me short and to the point answer only. do not add markdowns, etc)')
print(response.content)

297 calories 



# Search Engine

In [5]:
import requests
def googlesearch_results(query:str, number_of_results:int=10):
    query = "how many calories are in 100 grams of wheat flour roti in asia?"
    google_search_url_template = f"https://www.googleapis.com/customsearch/v1?key={google_search_project_api_key}&cx={google_search_project_id}&q={query}&num={number_of_results}&gl=pk&cr=countryPK&hl=en&lr=lang_en"
    response = requests.get(google_search_url_template)
    results = []
    for item in response.json()['items']:
        results.append(item['link'])
    return results


from duckduckgo_search import DDGS
def duckduckgo_results(query:str, num_results:int=10):
    results = DDGS().text(
        query+" filetype:html",
        max_results=num_results, region="pk"
    )
    result_list = []
    for result in results:
        result_list.append(result["href"])
    return result_list

# from googlesearch import search
# def googlesearch_results(query:str, num_results:int=10):
#     result_obj = search(
#         query, num_results=num_results,
#         lang="en"
#     )
#  
#     result_list = []
#     for i in result_obj:
#         result_list.append(i)
#     return result_list

gc.collect()

0

In [6]:
googlesearch_results("how many calories are in 100 grams of wheat flour roti in asia?:", 4)

['https://www.pc.gov.pk/uploads/report/Pakistan_Dietary_Nutrition_2019.pdf',
 'https://www.hofo.pk/buckwheat-101-nutrition-facts-and-health-benefits/',
 'https://himalayanchef.pk/blogs/blog/myths-about-rice',
 'https://www.foodnerd.pk/blogs/calories-in-roti']

In [7]:
duckduckgo_results("how many calories are in 100 grams of wheat flour roti in asia?", 6)

['https://www.irastoworldhealth.com/nutrition/calories-roti',
 'https://rotimatic.com/blogs/roti/calorie-of-roti',
 'https://www.fatsecret.com/calories-nutrition/generic/roti?portionid=333753&portionamount=100.000',
 'https://www.livestrong.com/article/324113-bajra-roti-nutrition/',
 'https://foodstruct.com/food/chapati',
 'https://www.livestrong.com/article/305496-the-calories-in-roti/']

# Text Loading

In [8]:
from langchain.document_loaders import WebBaseLoader

In [9]:
def load_webpages(website_url:list):
    """
    Load webpages from a list of urls

    INPUT:
        website_url: list of urls
    OUTPUT:
        langchain document object
    """
    loader = WebBaseLoader(website_url)
    return loader.load()

def text_cleaner(text:str):
    """
    Clean text from html tags, extra spaces, newlines, etc
    INPUT:
        text: string
    OUTPUT:
        cleaned_text: string
    """
    cleaned_text = re.sub(r"\n{3,}", "\n\n", text)
    cleaned_text = re.sub(r"\xa0|\r|\t", " ", cleaned_text)
    # cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    cleaned_text = re.sub(r"\s{2,}", " ", cleaned_text)
    cleaned_text = re.sub(r"<[^>]+>", "", cleaned_text)
    return cleaned_text

def langchain_document_cleaner(document_obj):
    """
    Clean page_content of langchain document object
    INPUT:
        document: langchain document object
    OUTPUT:
        cleaned_document: langchain document object
    """
    for i in range(len(document_obj)):
        document_obj[i].page_content = text_cleaner(document_obj[i].page_content)
    return document_obj

In [11]:
documents = load_webpages(duckduckgo_results("how many calories are in 100 grams of wheat flour roti in asia?", 10))
documents = langchain_document_cleaner(documents)
gc.collect()

21108

# Creating Chunks

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
# all-MiniLM-L12-v2 model has a limit of 256 words. ==> 4 x 256 = 1024 characters max
splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=200)
chunks = splitter.split_documents(documents)

# Embeddings

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2", cache_folder="temp")

gc.collect()

7

# Vector DB

In [15]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(chunks, embeddings)
db.save_local(folder_path="vdbs", index_name="temp_index_1")

In [16]:
db.similarity_search("how many calories are in 100 grams of wheat flour roti in asia?", k=2)

[Document(page_content='(Indian bread), wheat" indicates that 100 grams of roti has 299 calories. An 8-inch, or 52-gram piece has 155 calories. Ultimately, the difference comes down to slight variations in manufacturers and what they put in their roti. Advertisement We Recommend Nutrition Are All Beans Gluten-Free? Nutrition 20 Scary-Sounding Food Additives That Are Actually Harmless Nutrition Ready to Track Your Macros? Here’s Everything You Need to Know Traditionally roti is made with crushed atta flour from semi-hard durum wheat, a variety of wheat typically used to make pasta. Since atta comes from wheat, you should avoid roti or look for gluten-free varieties if you have gluten sensitivity. The other traditional ingredients are water and oil.', metadata={'source': 'https://www.livestrong.com/article/305496-the-calories-in-roti/', 'title': 'The Calories in Roti | livestrong', 'description': 'Roti nutrition and the calories in roti can vary based on family recipes and differences in

# Main Flow

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    '''You are an expert content writer and RAG specialist. Your task is to:
1. Rephrase the user query into a concise search string optimized for search engines.
2. Rephrase the query to capture the semantic intent for searching in a vector store.
3. *Always Return the result as a JSON object with keys: "original_query", "search_query", "vector_store_query".*

Example:
User Query: "I'm feeling tired all the time. What foods can help?"
Answer:
{{
    "original_query": "I'm feeling tired all the time. What foods can help?",
    "search_query": "foods to boost energy levels and reduce fatigue",
    "vector_store_query": "foods to help with fatigue"
}}

User Query: "{user_query}"
Answer:

    '''
)

# prompt = prompt_template.format(user_query=user_query)

In [18]:
import json
def string_to_json(string):
    try:
        string = string.replace('```json', '').replace('```', '').strip()
        json_obj = json.loads(string)
        return json_obj
    except json.JSONDecodeError:
        print("Invalid JSON string")
        return None

In [21]:
user_query = "how many calories are in 100 grams of wheat flour roti in asia?"

prompt = prompt_template.format(user_query=user_query)
llm_res = llm.invoke(prompt)
json_res = string_to_json(llm_res.content)

In [28]:
json_res

{'original_query': 'how many calories are in 100 grams of wheat flour roti in asia?',
 'search_query': 'calories in 100g wheat flour roti',
 'vector_store_query': 'calorie content of wheat roti, nutritional information of roti'}

In [31]:
similar_docs = db.similarity_search(json_res['vector_store_query'], k=5)

In [32]:
similar_docs

[Document(page_content='(Indian bread), wheat" indicates that 100 grams of roti has 299 calories. An 8-inch, or 52-gram piece has 155 calories. Ultimately, the difference comes down to slight variations in manufacturers and what they put in their roti. Advertisement We Recommend Nutrition Are All Beans Gluten-Free? Nutrition 20 Scary-Sounding Food Additives That Are Actually Harmless Nutrition Ready to Track Your Macros? Here’s Everything You Need to Know Traditionally roti is made with crushed atta flour from semi-hard durum wheat, a variety of wheat typically used to make pasta. Since atta comes from wheat, you should avoid roti or look for gluten-free varieties if you have gluten sensitivity. The other traditional ingredients are water and oil.', metadata={'source': 'https://www.livestrong.com/article/305496-the-calories-in-roti/', 'title': 'The Calories in Roti | livestrong', 'description': 'Roti nutrition and the calories in roti can vary based on family recipes and differences in